In [1]:
# Install Gradio for creating a simple web interface
# Install the latest version of Hugging Face Transformers (for GPT-2)
!pip install -q gradio
!pip install -q git+https://github.com/huggingface/transformers.git

Installing build dependencies ... done
Getting requirements to build wheel ... done
Preparing metadata (pyproject.toml) ... done


In [2]:
# Import Gradio for UI, TensorFlow backend for running GPT2, and the tokenizer/model from Transformers
import gradio as gr
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

In [3]:
# Load the GPT-2 tokenizer to convert input text into tokens
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Load the GPT-2 model and assign the end-of-sequence token for padding
# This model will be used to generate text
model = TFGPT2LMHeadModel.from_pretrained(
    "gpt2",
    pad_token_id=tokenizer.eos_token_id  # Ensures the model handles padding correctly
)

tokenizer_config.json: 100%|████████████████████████████████| 26.0/26.0 [00:00<00:00, 521B/s]
vocab.json: 100%|████████████████████████████████| 1.04M/1.04M [00:00<00:00, 4.06MB/s]
merges.txt: 100%|████████████████████████████████| 456k/456k [00:00<00:00, 5.41MB/s]
tokenizer.json: 100%|████████████████████████████████| 1.36M/1.36M [00:00<00:00, 14.0MB/s]
config.json: 100%|████████████████████████████████| 665/665 [00:00<00:00, 15.8kB/s]
model.safetensors: 100%|████████████████████████████████| 548M/548M [00:14<00:00, 67.9MB/s]
All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [4]:
# Define a function that accepts user input and generates a text paragraph
def generate_text(input_text):
    # Convert the input text into model-readable format (tensor of tokens)
    input_ids = tokenizer.encode(input_text, return_tensors='tf')

    # Generate new text using beam search (more coherent than greedy or random)
    beam_output = model.generate(
        input_ids,
        max_length=350,              # Maximum length of the generated text
        num_beams=5,                 # Beam search with 5 beams (better quality)
        no_repeat_ngram_size=2,     # Avoid repeating 2-word phrases
        early_stopping=True         # Stop when output seems complete
    )

    # Decode the generated output into human-readable text
    output = tokenizer.decode(
        beam_output[0],
        skip_special_tokens=True,   # Remove tokens like
        clean_up_tokenization_spaces=True  # Clean extra spaces
    )

    # Return only full sentences, cleaned up
    return ".".join(output.split(".")) + "."

In [5]:
# Define a textbox output component
output_text = gr.Textbox()

# Create and launch the Gradio interface
gr.Interface(
    fn=generate_text,         # Function to run on input
    inputs="textbox",         # Input component (user prompt)
    outputs=output_text,      # Output component (generated paragraph)
    title="🧠 GENERATIVE TEXT MODEL (GPT-2)"  # Interface title
).launch()